In [ ]:
# https://raki-1203.github.io/boostcamp_ai_tech/week_9/01.-Introduce-BERT-Language-Model/

# ETRI KoBERT : 형태소 단위로 분리(ETRI 형태소 분석기 사용해야함) -> WordPieceTokenizer
# KoBERT : WordPieceTokenizer만 사용
#   형태소 분석기를 태우고 sub-word tokenizer 를 사용한게 성능이 가장 좋았대
# --------------------
# Advanced BERT model - KBQA 에서 가장 중요한 entity 정보가 기존 BERT 에서는 무시

# BertWordPieceTokenizer : BERT에서 사용된 워드피스 토크나이저(WordPiece Tokenizer)
# CharBPETokenizer : 오리지널 BPE
# ByteLevelBPETokenizer : BPE의 바이트 레벨 버전
# SentencePieceBPETokenizer : 앞서 본 패키지 센텐스피스(SentencePiece)와 호환되는 BPE 구현체


# https://velog.io/@taekkim/Hugginface-transformer-%EC%84%A4%EA%B3%84%EA%B5%AC%EC%A1%B0-%EC%82%B4%ED%8E%B4%EB%B3%B4%EA%B8%B0
# 내가 사용할 모델이 어떤 토크나이저 사용하는지 확인후 MODEL NAME만 바꿔주면됨
# tokenize할 때에 padding, truncation 등 다양한 옵션을 설정할 수 있으며, 모델이 어떤 프레임워크를 사용하는가(Tensorflow 또는 PyTorch)에 따라 input 타입을 변경 시켜주는 return_tensors 인자도 있음
# model을 사용할 때 명시했던 것과 동일한 ID로 tokenizer를 생성해야함
# tokenizer 또한 직접 명시하여 내가 사용할 것을 지정해 주거나, (bert-base-cased, bert-base-uncassed, bert-large-cased)
#   AutoTokenizer를 사용하여 이미 구비된 model에 알맞는 tokenizer를 자동으로 불러올 수도 있음


In [ ]:
# from tokenizers import BertWordPieceTokenizer

# tokenizer = BertWordPieceTokenizer(lowercase=False, trip_accents=False)

# vocab_size = 30000
# limit_alphabet = 6000
# min_frequency = 5

# files : 단어 집합을 얻기 위해 학습할 데이터
# vocab_size : 단어 집합의 크기
# limit_alphabet : 병합 전의 초기 토큰의 허용 개수.
# min_frequency : 최소 해당 횟수만큼 등장한 쌍(pair)의 경우에만 병합 대상이 된다.

# tokenizer.train(files=ndf, vocab_size=vocab_size, limit_alphabet=limit_alphabet, min_frequency=min_frequency)

# tokenizer.save_model('./')

# encoded = tokenizer.encode('테스트할 문장이 들어갈 자리인가바')
# print('토큰화 결과 : ', encoded.tokens)
# print('정수 인코딩 : ', endcoded.ids)
# print('디코딩 : ', tokenizer.decode(encoded.ids))

In [ ]:
# train, test dataset 분리해야함
# dataloader 만들기
# https://americanoisice.tistory.com/82

# 데이터로더 안만들고 그냥 tsv로 변환해서 불러옴
#https://velog.io/@dev-junku/KoBERT-%EB%AA%A8%EB%8D%B8%EC%97%90-%EB%8C%80%ED%95%B4

# 이런식으로 넣어줘도됨
#https://velog.io/@danbibibi/KoBERT-fine-tuning-Sentiment-Analysis


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/MyDrive"

Mounted at /content/drive
/content/drive/MyDrive


In [ ]:
%cd '/content/drive/MyDrive/dataset/dataset'


import os
import pandas as pd 
import numpy as np

df = pd.read_csv('dataset_clean.csv')

/content/drive/.shortcut-targets-by-id/19AmDlGex-hSZJyofrcVWB6uu3yQV-PSB/dataset/dataset


In [ ]:
df = df.drop(columns='Unnamed: 0')
df

,content,label
0,ㅇㄱㄹㅇㅠ,0
1,왜 200명을 몰살해 ㅋㅋㅋ,0
2,모인독고 ㅁㅈㅎ,0
3,안돼 집사고 융자 40퍼 이하로 내려가면 사라,0
4,와 중궈 말빨 ㄷㄷㄷ,0
...,...,...
511597,와꾸는 솔직히 1번이 우측 두년 압살이다.,1
511598,강원도는 그래도 미통당이 더많이 이겼는데,0
511599,조회수 400따리면 지역 동수준 맘카페도 아닌데 뭘..,0
511600,족밥이네 꼴랑 1000베 거냐 졸보새끼야2500베 못가면 계정폭파함,0


In [ ]:
from sklearn.model_selection import train_test_split

x = df['content']
y = df['label']

x

0                                         ㅇㄱㄹㅇㅠ
1                               왜 200명을 몰살해 ㅋㅋㅋ
2                                      모인독고 ㅁㅈㅎ
3                     안돼 집사고 융자 40퍼 이하로 내려가면 사라
4                                   와 중궈 말빨 ㄷㄷㄷ
                          ...                  
511597                  와꾸는 솔직히 1번이 우측 두년 압살이다.
511598                   강원도는 그래도 미통당이 더많이 이겼는데
511599           조회수 400따리면 지역 동수준 맘카페도 아닌데 뭘..
511600    족밥이네 꼴랑 1000베 거냐 졸보새끼야2500베 못가면 계정폭파함
511601                                     노다ㅓㅂ
Name: content, Length: 511602, dtype: object

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, shuffle=True, random_state=42)

x_train

349140                4월1일 tv조선 밤9시 뉴스
377087           전두환 반란군놈의 새끼 피셜을 믿고있노
203996                        아마 부연당에.
161349                             때라.
248605                          노토전생ㅋㅋ
                      ...             
259178                     백두산권총 줍 인증!
365838                           무현색ㅋㅋ
131932                      공무원새끼들 미쳤네
146867                            븅딱아.
121958    골수좌파들이 교육받는거가 의식적으로 저렇게 웃는거다
Name: content, Length: 383701, dtype: object

In [ ]:
# y_train

In [ ]:
# x_test

In [ ]:
# !pip install git+https://git@github.com/SKTBrain/KoBERT.git@master


In [ ]:
%cd /content/drive/MyDrive/KoBERT

/content/drive/MyDrive/KoBERT


In [ ]:
# https://americanoisice.tistory.com/82

!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 KB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 KB 15.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 72.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 56.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 881.9/881.9 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 98.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 KB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 101.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 68.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [ ]:
!pip install boto3
!pip install transformers
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW # 인공지능 모델의 초기값 지정 함수를 아담으로 지정한다.
from transformers.optimization import get_cosine_schedule_with_warmup

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os

n_devices = torch.cuda.device_count()
print(n_devices)

for i in range(n_devices):
    print(torch.cuda.get_device_name(i))

1
Tesla T4


In [ ]:
#GPU 사용
device = torch.device('cuda:0')

In [ ]:
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. /content/drive/MyDrive/KoBERT/.cache/kobert_v1.zip
using cached model. /content/drive/MyDrive/KoBERT/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
# Setting parameters 필수
max_len = 150
batch_size = 32
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 100
learning_rate =  5e-5

In [ ]:
dataset_train = pd.concat([x_train, y_train], axis=1)

In [ ]:
dataset_test = pd.concat([x_test, y_test], axis=1)

In [ ]:
for i, data in enumerate(dataset_train['label']):
  if type(data) != int:
    print(i)

In [ ]:
# !wget https://www.dropbox.com/s/374ftkec978br3d/ratings_train.txt?dl=1
# !wget https://www.dropbox.com/s/977gbwh542gdy94/ratings_test.txt?dl=1

In [ ]:
train = dataset_train.reset_index(drop=True)
test = dataset_test.reset_index(drop=True)

In [ ]:
train.to_csv('train.tsv', sep='\t', encoding='utf-8', index=False)
test.to_csv('test.tsv', sep='\t', encoding='utf-8', index=False)

In [ ]:
train = nlp.data.TSVDataset("train.tsv", field_indices=[0,1], num_discard_samples=1)
test = nlp.data.TSVDataset("test.tsv", field_indices=[0,1], num_discard_samples=1)

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        length = len(dataset)
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))  

In [ ]:
# 토큰화
# https://velog.io/@dev-junku/KoBERT-%EB%AA%A8%EB%8D%B8%EC%97%90-%EB%8C%80%ED%95%B4

tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(test, 0, 1, tok, max_len, True, False)

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)


using cached model. /content/drive/MyDrive/KoBERT/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
data_train[0]

(array([   2,  605, 7028,   93, 7126,  517,  442,  453, 7256, 2265,  234,
        6705, 1542,    3,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1, 

In [ ]:
class BERTClassifier(nn.Module): ## 클래스를 상속
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [ ]:
#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)



In [ ]:
#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

In [ ]:
#train
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    
    PATH = '/content/drive/MyDrive/KoBERT'  

    if os.access(PATH, os.W_OK):
        print("쓰기 권한 있음")
    else:
        print("쓰기 권한 없음")
    checkpoint_name = f"checkpoint_epoch{e+1}.pt"
    torch.save(
        {
            "epoch": e+1,
            "model_state_dict": model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
            "scheduler_state_dict": scheduler.state_dict(),
            "loss": loss,
        },
        PATH + checkpoint_name,
    )


<ipython-input-35-47e4881ffad5>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/11991 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 0.6828019618988037 train acc 0.53125
epoch 1 batch id 101 loss 0.725944995880127 train acc 0.5631188118811881
epoch 1 batch id 201 loss 0.7625330686569214 train acc 0.5688743781094527
epoch 1 batch id 301 loss 0.6889747381210327 train acc 0.5830564784053156
epoch 1 batch id 401 loss 0.6694361567497253 train acc 0.5915679551122195
epoch 1 batch id 501 loss 0.586918294429779 train acc 0.6029815369261478
epoch 1 batch id 601 loss 0.5853334069252014 train acc 0.6205282861896838
epoch 1 batch id 701 loss 0.4813894033432007 train acc 0.6358773181169758
epoch 1 batch id 801 loss 0.4767425060272217 train acc 0.6486033083645443
epoch 1 batch id 901 loss 0.38161197304725647 train acc 0.6576026637069923
epoch 1 batch id 1001 loss 0.4749530553817749 train acc 0.66624000999001
epoch 1 batch id 1101 loss 0.3921597898006439 train acc 0.6741030881017257
epoch 1 batch id 1201 loss 0.5059137344360352 train acc 0.6811511240632806
epoch 1 batch id 1301 loss 0.4374862313270569 train

<ipython-input-35-47e4881ffad5>:24: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/3997 [00:00<?, ?it/s]

...

epoch 4 test acc 0.8293696026330093
쓰기 권한 있음


  0%|          | 0/11991 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.5219752192497253 train acc 0.875
epoch 5 batch id 101 loss 0.03162671998143196 train acc 0.9232673267326733
epoch 5 batch id 201 loss 0.5486136674880981 train acc 0.9137126865671642
epoch 5 batch id 301 loss 0.33167362213134766 train acc 0.9136212624584718
epoch 5 batch id 401 loss 0.3909931480884552 train acc 0.9140430174563591
epoch 5 batch id 501 loss 0.16046389937400818 train acc 0.9129865269461078
epoch 5 batch id 601 loss 0.0906793475151062 train acc 0.9125415973377704
epoch 5 batch id 701 loss 0.11918327212333679 train acc 0.9136055634807418
epoch 5 batch id 801 loss 0.2363058477640152 train acc 0.9131164169787765
epoch 5 batch id 901 loss 0.13467355072498322 train acc 0.9121462264150944
epoch 5 batch id 1001 loss 0.07929347455501556 train acc 0.9126810689310689
epoch 5 batch id 1101 loss 0.21782919764518738 train acc 0.9135445049954587
epoch 5 batch id 1201 loss 0.21451637148857117 train acc 0.913171315570358
epoch 5 batch id 1301 loss 0.21538366377353

  0%|          | 0/3997 [00:00<?, ?it/s]

epoch 5 test acc 0.8304180721748208
쓰기 권한 있음


In [ ]:
## 학습 모델 저장
PATH = '/content/drive/MyDrive/KoBERT/' # google 드라이브 연동 해야함. 관련코드는 뺐음
torch.save(model, PATH + 'check.pth')  # 전체 모델 저장
torch.save(model.state_dict(), PATH + 'model_state_dict.pth')  # 모델 객체의 state_dict 저장
torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict()
}, PATH + 'all.tar')  # 여러 가지 값 저장, 학습 중 진행 상황 저장을 위해 epoch, loss 값 등 일반 scalar값 저장 가능

In [ ]:
#test
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

model.eval()
output=[]
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(test_dataloader)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)

    valid_length= valid_length
    label = label.long().to(device)

    out = model(token_ids, valid_length, segment_ids)

    for i in out:
        logits=i
        logits = logits.detach().cpu().numpy()
        output.append(logits)

In [ ]:
result = np.argmax(output, axis=1)
result

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-37-3bbdc6f6be4b>", line 1, in <cell line: 1>
    result = np.argmax(output, axis=1)
NameError: name 'output' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/IPython/core/ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/usr/local/lib/python3.9/dist-packages/IPytho

In [ ]:
# # https://wikidocs.net/166801
# # 전체 데이터셋이 아닌 개별 배치(batch)에 대해서 별도로 패딩(padding)을 수행하여 과도하게 긴 입력으로 인한 과도한 패딩(padding) 작업을 방지

# from transformers import DataCollatorWithPadding

# data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
